DEPENDENCIAS

In [8]:
#dependencias

import cv2
import numpy as np
import face_recognition as fr
import os
from datetime import datetime
import random

ACCEDER A LA CARPETA CON LAS IMGS -> "BASE DE DATOS"

In [9]:
#Acceso a la carpeta
path = '/home/jorge/Documentos/Sistema-Reconocimiento-Facial-Instalaciones/ML/fotos'

images = [] #Lista de imagenes
clases = [] #Lista de nombres
lista = os.listdir(path) #Lista de archivos en la carpeta

comp1 = 100 #Umbral de comparacion


# Declaración de color verde
color_verde = (0, 255, 0)  # R, G, B para verde
# Declaracion de color rojo
color_rojo = (0, 0, 255)  # R, G, B para rojo
print(lista)

['Andres.jpg', 'Giovanny.jpg']


LEER LOS ROSTROS DE LA DDBB

In [10]:
for lis in lista:
    # leer las imagenes de los rostros
    imgdb = cv2.imread(f'{path}/{lis}') #Leer la imagen
    # almacernar las imagenes en una lista
    images.append(imgdb)
    #almacenar el nombre de la imagen en una lista
    clases.append(os.path.splitext(lis)[0]) #almacenar el nombre de la imagen sin la extension
print(clases)


['Andres', 'Giovanny']


CODIFICACION DE LOS ROSTROS

In [11]:
#Funcion para codificar las imagenes
def codRostros(images):
    listCod = []
    
    # iterar las imagenes
    for img in images:
        # convertir a escala de grises
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # codificar la imagen
        cod = fr.face_encodings(img)[0]
        # almacenar la codificacion en una lista
        listCod.append(cod)
    return listCod
    
""" # Función para codificar las imágenes
def codRostros(images):
    listCod = []
    
    # Iterar las imágenes
    for img in images:
        # Convertir a escala de grises
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Codificar la imagen
        face_encodings = fr.face_encodings(img)
        
        if len(face_encodings) > 0:
            # Si se encontró al menos un rostro en la imagen, obtener su codificación
            cod = face_encodings[0]
            # Almacenar la codificación en la lista
            listCod.append(cod)
        else:
            # Si no se encontraron rostros en la imagen, puedes manejarlo de alguna manera
            # Por ejemplo, puedes imprimir un mensaje o simplemente continuar con la siguiente imagen.
            print("No se encontraron rostros en la imagen.")

    return listCod """


' # Función para codificar las imágenes\ndef codRostros(images):\n    listCod = []\n    \n    # Iterar las imágenes\n    for img in images:\n        # Convertir a escala de grises\n        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n        \n        # Codificar la imagen\n        face_encodings = fr.face_encodings(img)\n        \n        if len(face_encodings) > 0:\n            # Si se encontró al menos un rostro en la imagen, obtener su codificación\n            cod = face_encodings[0]\n            # Almacenar la codificación en la lista\n            listCod.append(cod)\n        else:\n            # Si no se encontraron rostros en la imagen, puedes manejarlo de alguna manera\n            # Por ejemplo, puedes imprimir un mensaje o simplemente continuar con la siguiente imagen.\n            print("No se encontraron rostros en la imagen.")\n\n    return listCod '

REGISTRAR HORA DEL RECONOCIMIENTO -> INGRESO

In [12]:
#Abriel el archivo en modo lectura y escritura -> archivo csv donde se guarda el registro de hora de entrada y salida

import time  # Importa el módulo time

# Declaración de variables globales
ultima_entrada = None  # Variable para registrar la hora de la última entrada

# Función para registrar la hora
def horario(nombre):
    global ultima_entrada  # Declarar la variable global
    with open('Horario.csv', 'a') as h:  # 'a' para abrir en modo append (agregar al archivo)
        info = datetime.now()  # obtener la hora actual
        # extraer la fecha
        fecha = info.strftime('%Y:%m:%d')
        # extraer la hora
        hora = info.strftime('%H:%M:%S')
        
        # Verificar si ha pasado suficiente tiempo desde la última entrada
        if ultima_entrada is None or (info - ultima_entrada).total_seconds() > 60:  # Por ejemplo, 60 segundos de espera
            # guardar la información en el archivo
            h.writelines(f'\n{nombre},{fecha},{hora},Entrada')
            print(info)
            ultima_entrada = info  # Actualiza la hora de la última entrada


LLAMADO DE FUNCIONES

In [13]:
# llamar la funcion para codificar las imagenes
rostrosCOD = codRostros(images)

#llamar la camara
cap = cv2.VideoCapture(0)         

LEER EL PROGRAMA -> CAMARA -> FUNCIONES ETC

In [14]:
while True:
    # leer los frames de la camara
    ret, frame = cap.read()
    # redimencionar las imagenes
    frame2 = cv2.resize(frame, (0,0), None, 0.25, 0.25)
    
    # convertir a escala de grises
    rgb = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
    
    #buscar los rostros en la imagen
    faces = fr.face_locations(rgb)
    facesCod = fr.face_encodings(rgb, faces)
    
    # Inicializar la variable para rastrear si se detectó un rostro conocido
    rostro_conocido = False
    
    # iterar los rostros
    for faceCod, faceloc in zip(facesCod, faces):
        # comparar los rostros de la DDBB con la camara
        comparacion = fr.compare_faces(rostrosCOD, faceCod)
        
        # calcular la coincidencia de imagen camara
        #simi = fr.face_distance(rostrosCOD, faces)
        simi = fr.face_distance(rostrosCOD, faceCod)
        
        # buscar el valor más bajo -> mayor coincidencia
        min = np.argmin(simi)
        
        if comparacion[min]:
            # comparar la coincidencia con el umbral
            nombre = clases[min]#obtener el nombre de la persona
            print(nombre)
            
            # obtener las coordenadas
            y1, x2, y2, x1 = faceloc
            
            # redimencionar las coordenadas
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            
            # obtener el indice 
            indice = comparacion.index(True)

            # establecer el color verde
            r, g, b = color_verde

            # cambiar el color de la caja a verde
            cv2.rectangle(frame, (x1,y1), (x2,y2), (r,g,b), 3) #rectángulo para el rostro
            cv2.rectangle(frame, (x1,y2-35), (x2,y2), (r,g,b), cv2.FILLED) #rectángulo para el nombre
            cv2.putText(frame, nombre, (x1+6,y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2) #texto para el nombre

            entrada = horario(nombre) #llamar la función para registrar la hora
            print(f'hora de ingreso {entrada}')
            # Marcar que se detectó un rostro conocido
            rostro_conocido = True
        else:
            # Si no se detecta un rostro conocido, pintar un cuadro rojo con "desconocido"
            r, g, b = color_rojo
            cv2.rectangle(frame, (x1,y1), (x2,y2), (r,g,b), 3) # Rectángulo para el rostro
            cv2.rectangle(frame, (x1,y2-35), (x2,y2), (r,g,b), cv2.FILLED) # Rectángulo para "desconocido"
            cv2.putText(frame, "Desconocido", (x1+6,y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2) # Texto para "desconocido"
            rostro_conocido = False
            
    # mostrar Frame
    cv2.imshow('RECONOCIMIENTO FACIAL', frame)

    # leer el teclado
    t = cv2.waitKey(1)
    if t == 27:
        break          
# Liberar la cámara y cerrar la ventana
cap.release()
cv2.destroyAllWindows()

Giovanny
2023-10-26 14:52:24.315199
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora de ingreso None
Giovanny
hora